In [1]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [2]:
# Display the shapes of the data.
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (50000, 32, 32, 3)
Shape of X_test: (10000, 32, 32, 3)
Shape of y_train: (50000, 1)
Shape of y_test: (10000, 1)


In [3]:
from layers_activation import ReluLayer
from layers_convolution import ConvolutionLayer
from layers_pooling import PoolLayer
from layers_flatten import Flatten
from layers_dense import DenseLayer
from loss import SoftmaxLayer
from network import Model

In [4]:
model = Model(
    ConvolutionLayer(filters=3, filter_size=3, padding=1, stride=1),
    ReluLayer(),
    PoolLayer(filter_size=2, stride=2, mode='max'),
    Flatten(),
    DenseLayer(units=10),
    SoftmaxLayer()
)

In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
y_train.shape

(50000, 1)

In [7]:
# Downsize the training data in the interest of time
X_train = X_train[ :500, :, :, :]
y_train = y_train[ :500, :]

In [8]:
model.train(X_train, y_train, 0.003, batch_size=256, epochs=1)

Running epoch: 1
A_prev input shape in Convolution forward prop: (256, 32, 32, 3)
W weights input shape in Convolution forward prop: (3, 3, 3, 3)
b biases input shape in Convolution forward prop: (1, 1, 1, 3)
Z output shape in Convolution forward prop: (256, 32, 32, 3)
A_prev input shape in Pooling forward: (256, 32, 32, 3)
A output shape in Pooling forkward: (256, 16, 16, 3)
Z input shape in Flatten forward prop: (256, 16, 16, 3)
data output shape in Flatten forward prop: (768, 256)
X input shape in Dense forward prop: (768, 256)
W weights input shape in Dense forward prop: (10, 768)
b biases input shape in Dense forward prop: (10, 1)
mini_batch[0] shape (256, 32, 32, 3)
mini_batch_true_labels shape (256, 10)
mini_batch_preds final shape (10, 256)
dA.shape: (10, 256)
dA input shape in Softmax backward: (10, 256)
Z input shape in Softmax backward: (10, 256)
dZ input shape in Dense backward: (10, 256)
out output shape in Dense backward: (768, 256)
Z input shape in Flatten backward: (768

In [9]:
X_test.shape

(10000, 32, 32, 3)

In [10]:
X_test = X_test[:100, :, :, :]

In [11]:
predictions = model.predict(X_test)

A_prev input shape in Convolution forward prop: (100, 32, 32, 3)
W weights input shape in Convolution forward prop: (3, 3, 3, 3)
b biases input shape in Convolution forward prop: (1, 1, 1, 3)
Z output shape in Convolution forward prop: (100, 32, 32, 3)
A_prev input shape in Pooling forward: (100, 32, 32, 3)
A output shape in Pooling forkward: (100, 16, 16, 3)
Z input shape in Flatten forward prop: (100, 16, 16, 3)
data output shape in Flatten forward prop: (768, 100)
X input shape in Dense forward prop: (768, 100)
W weights input shape in Dense forward prop: (10, 768)
b biases input shape in Dense forward prop: (10, 1)


In [12]:
predictions = predictions.T

In [13]:
predictions[:5]

array([[0.09866571, 0.09458959, 0.09458459, 0.09484569, 0.08941313,
        0.10330677, 0.08814378, 0.10387207, 0.10303562, 0.12954305],
       [0.09984734, 0.09427723, 0.09598766, 0.09477925, 0.09102968,
        0.10315467, 0.09002886, 0.09987761, 0.10589395, 0.12512375],
       [0.09982463, 0.09748786, 0.09135531, 0.09810533, 0.08835648,
        0.10776814, 0.08881119, 0.09981378, 0.09946381, 0.1290135 ],
       [0.09818279, 0.09539156, 0.0950155 , 0.09515497, 0.08980819,
        0.10243714, 0.08860699, 0.1031823 , 0.10176492, 0.13045564],
       [0.09926763, 0.09616122, 0.09413724, 0.09861349, 0.08985723,
        0.10155853, 0.08860726, 0.09725155, 0.10535321, 0.12919264]])

In [14]:
import numpy as np 
np.argmax(predictions, axis=1)[:5]

array([9, 9, 9, 9, 9])

In [15]:
def to_categorical(labels, num_classes):
    """
    Converts image labels to one-hot encoded binary vectors with length of num_classes.
    Arguments:
        labels -- image classification labels e.g. 1 = 'airplane', nested numpy arrays of integers.
        num_classes -- number of classes in image classification task, integer.
    Returns:
        label_vec -- binary vector representation of the class, 1 for correct label, 0's otherwise. 
    """
    # Initialize a zero matrix with the correct dimensions.
    label_vec = np.zeros((len(labels), num_classes))
    # Loop through the labels in the input.
    for i in range(len(labels)):
        # Create an new index for binary encoding, subtracting 1. 
        # An image with label [6] on a scale of 1-10, will have a 1 at index 5 in the vector.
        new_index = labels[i][0] - 1
        # Replace the 0 at the given location with a 1 for the correct class.
        label_vec[i][new_index] = 1
    return label_vec

In [16]:
y_test = to_categorical(y_test, 10)

In [17]:
y_test[:5]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [18]:
np.argmax(y_test, axis=1)[:5]

array([2, 7, 7, 9, 5])

In [19]:
arr = [[1, 2, 3, 4]]
np.argamax(arr, )

AttributeError: module 'numpy' has no attribute 'argamax'

In [ ]:
import numpy as np 
Y_hat_temp = np.array([8, 6, 0, 8, 6])
Y_temp = np.array([5, 8, 8, 3, 0])

In [ ]:
(Y_hat_temp == Y_temp).mean()